# Transfer Learning Models

While some transfer learning models did have greater evaluation metrics, mainly recall, on average. No models outperformed the custom 2x2C1D with Dropout model from 004_Modeling_Local.ipynb.

## Libraries to Import

In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from PIL import Image
import keras
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization

import os
import gc
from timeit import default_timer as timer

## Define Image Directories

In [2]:
train_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/training/'
test_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/testing/'
val_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/validation/'

Create flow_from_directory objects for Keras model. Using a downsized dataset with class balance for local model running.

In [3]:
train_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(train_dir,
                                                                  target_size = (256,256), 
                                                                  color_mode='rgb',
                                                                  class_mode='binary')
val_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(val_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')
test_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(test_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')

Found 1346 images belonging to 2 classes.
Found 593 images belonging to 2 classes.
Found 947 images belonging to 2 classes.


Xception has 22,910,480 trainable parameters, and a top-5 accuracy of 0.945.

## Xception -- 5 epochs

In [19]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

83689472/83683744 [==============================] - 3s 0us/step


In [22]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [23]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 140s 3s/step - loss: 0.6371 - accuracy: 0.6612 - recall: 0.6657 - val_loss: 0.5866 - val_accuracy: 0.7285 - val_recall: 0.7381
Epoch 2/5
43/43 [==============================] - 157s 4s/step - loss: 0.5689 - accuracy: 0.7348 - recall: 0.7504 - val_loss: 0.5515 - val_accuracy: 0.7369 - val_recall: 0.7517
Epoch 3/5
43/43 [==============================] - 149s 3s/step - loss: 0.5432 - accuracy: 0.7422 - recall: 0.7221 - val_loss: 0.5315 - val_accuracy: 0.7521 - val_recall: 0.7789
Epoch 4/5
43/43 [==============================] - 147s 3s/step - loss: 0.5259 - accuracy: 0.7600 - recall: 0.7340 - val_loss: 0.5498 - val_accuracy: 0.7167 - val_recall: 0.8946
Epoch 5/5
43/43 [==============================] - 145s 3s/step - loss: 0.5078 - accuracy: 0.7615 - recall: 0.7563 - val_loss: 0.5155 - val_accuracy: 0.7589 - val_recall: 0.7891
Total Time Elapsed:  12  minutes  34  seconds


In [24]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 65s 2s/step - loss: 0.9548 - accuracy: 0.4340 - recall: 0.2685


## Xception -- 15 epochs

In [5]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [6]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [7]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 142s 3s/step - loss: 0.6473 - accuracy: 0.6367 - recall: 0.6865 - val_loss: 0.6001 - val_accuracy: 0.7403 - val_recall: 0.5782
Epoch 2/15
43/43 [==============================] - 152s 4s/step - loss: 0.5757 - accuracy: 0.7318 - recall: 0.6909 - val_loss: 0.5587 - val_accuracy: 0.7302 - val_recall: 0.7959
Epoch 3/15
43/43 [==============================] - 150s 3s/step - loss: 0.5401 - accuracy: 0.7511 - recall: 0.7251 - val_loss: 0.5347 - val_accuracy: 0.7504 - val_recall: 0.7823
Epoch 4/15
43/43 [==============================] - 148s 3s/step - loss: 0.5240 - accuracy: 0.7541 - recall: 0.7400 - val_loss: 0.5265 - val_accuracy: 0.7521 - val_recall: 0.7891
Epoch 5/15
43/43 [==============================] - 148s 3s/step - loss: 0.5080 - accuracy: 0.7689 - recall: 0.7578 - val_loss: 0.5233 - val_accuracy: 0.7504 - val_recall: 0.8163
Epoch 6/15
43/43 [==============================] - 147s 3s/step - loss: 0.5003 - accuracy: 0.7704 - reca

In [8]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 68s 2s/step - loss: 1.0511 - accuracy: 0.4636 - recall: 0.2778


VGG16 has 138,357,544 trainable parameters, and a top-5 accuracy of 0.901.

## VGG16 -- 5 epochs

In [5]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [6]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [7]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 322s 7s/step - loss: 0.6880 - accuracy: 0.5572 - recall: 0.3314 - val_loss: 0.6756 - val_accuracy: 0.7150 - val_recall: 0.7721
Epoch 2/5
43/43 [==============================] - 319s 7s/step - loss: 0.6673 - accuracy: 0.6709 - recall: 0.6256 - val_loss: 0.6589 - val_accuracy: 0.7184 - val_recall: 0.8333
Epoch 3/5
43/43 [==============================] - 316s 7s/step - loss: 0.6510 - accuracy: 0.7065 - recall: 0.6642 - val_loss: 0.6443 - val_accuracy: 0.7234 - val_recall: 0.7925
Epoch 4/5
43/43 [==============================] - 335s 8s/step - loss: 0.6362 - accuracy: 0.7065 - recall: 0.7073 - val_loss: 0.6331 - val_accuracy: 0.7302 - val_recall: 0.7619
Epoch 5/5
43/43 [==============================] - 312s 7s/step - loss: 0.6255 - accuracy: 0.7147 - recall: 0.7132 - val_loss: 0.6225 - val_accuracy: 0.7319 - val_recall: 0.7619
Total Time Elapsed:  27  minutes  12  seconds


In [8]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 148s 5s/step - loss: 0.7729 - accuracy: 0.4234 - recall: 0.2238


## VGG16 -- 15 epochs

In [9]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [10]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [11]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 309s 7s/step - loss: 0.6808 - accuracy: 0.6263 - recall: 0.7831 - val_loss: 0.6708 - val_accuracy: 0.6071 - val_recall: 0.8878
Epoch 2/15
43/43 [==============================] - 312s 7s/step - loss: 0.6611 - accuracy: 0.6471 - recall: 0.7385 - val_loss: 0.6565 - val_accuracy: 0.6779 - val_recall: 0.8095
Epoch 3/15
43/43 [==============================] - 313s 7s/step - loss: 0.6473 - accuracy: 0.6857 - recall: 0.7207 - val_loss: 0.6440 - val_accuracy: 0.7133 - val_recall: 0.7483
Epoch 4/15
43/43 [==============================] - 314s 7s/step - loss: 0.6364 - accuracy: 0.7058 - recall: 0.6374 - val_loss: 0.6365 - val_accuracy: 0.6813 - val_recall: 0.8197
Epoch 5/15
43/43 [==============================] - 312s 7s/step - loss: 0.6262 - accuracy: 0.7036 - recall: 0.7207 - val_loss: 0.6291 - val_accuracy: 0.6728 - val_recall: 0.8265
Epoch 6/15
43/43 [==============================] - 319s 7s/step - loss: 0.6147 - accuracy: 0.7043 - reca

In [12]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 151s 5s/step - loss: 0.8086 - accuracy: 0.4615 - recall: 0.3395


InceptionResNetV2 has 55,873,736 trainable parameters, and a top-5 accuracy of 0.953.

## InceptionResNetV2 -- 5 epochs

In [4]:
base_model = keras.applications.InceptionResNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

219062272/219055592 [==============================] - 8s 0us/step


In [5]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [6]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 156s 4s/step - loss: 0.6612 - accuracy: 0.6040 - recall: 0.6270 - val_loss: 0.6085 - val_accuracy: 0.6155 - val_recall: 0.9184
Epoch 2/5
43/43 [==============================] - 155s 4s/step - loss: 0.6047 - accuracy: 0.6932 - recall: 0.6909 - val_loss: 0.5432 - val_accuracy: 0.7774 - val_recall: 0.8027
Epoch 3/5
43/43 [==============================] - 154s 4s/step - loss: 0.5666 - accuracy: 0.7266 - recall: 0.7192 - val_loss: 0.5370 - val_accuracy: 0.8061 - val_recall: 0.6837
Epoch 4/5
43/43 [==============================] - 154s 4s/step - loss: 0.5538 - accuracy: 0.7221 - recall: 0.6954 - val_loss: 0.5089 - val_accuracy: 0.7555 - val_recall: 0.8571
Epoch 5/5
43/43 [==============================] - 157s 4s/step - loss: 0.5404 - accuracy: 0.7340 - recall: 0.7251 - val_loss: 0.4899 - val_accuracy: 0.8128 - val_recall: 0.8027
Total Time Elapsed:  13  minutes  17  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 74s 2s/step - loss: 0.8767 - accuracy: 0.4657 - recall: 0.2963


## InceptionResNetV2 -- 15 epochs

In [8]:
base_model = keras.applications.InceptionResNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [9]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [10]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 163s 4s/step - loss: 0.6693 - accuracy: 0.6129 - recall: 0.5795 - val_loss: 0.6048 - val_accuracy: 0.6374 - val_recall: 0.9252
Epoch 2/15
43/43 [==============================] - 158s 4s/step - loss: 0.5991 - accuracy: 0.7013 - recall: 0.6969 - val_loss: 0.5703 - val_accuracy: 0.7437 - val_recall: 0.5408
Epoch 3/15
43/43 [==============================] - 158s 4s/step - loss: 0.5762 - accuracy: 0.6961 - recall: 0.6999 - val_loss: 0.5371 - val_accuracy: 0.7285 - val_recall: 0.9048
Epoch 4/15
43/43 [==============================] - 159s 4s/step - loss: 0.5621 - accuracy: 0.7296 - recall: 0.7251 - val_loss: 0.5015 - val_accuracy: 0.8061 - val_recall: 0.7823
Epoch 5/15
43/43 [==============================] - 158s 4s/step - loss: 0.5442 - accuracy: 0.7325 - recall: 0.7325 - val_loss: 0.4957 - val_accuracy: 0.8094 - val_recall: 0.7619
Epoch 6/15
43/43 [==============================] - 159s 4s/step - loss: 0.5264 - accuracy: 0.7370 - reca

In [11]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 74s 2s/step - loss: 0.8598 - accuracy: 0.4879 - recall: 0.3534


MobileNetV2 has 3,538,984 trainable parameters, and a top-5 accuracy of 0.901.

## MobileNetV2 -- 5 epochs

In [4]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [5]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [6]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 43s 994ms/step - loss: 0.6419 - accuracy: 0.6308 - recall: 0.5884 - val_loss: 0.5471 - val_accuracy: 0.7740 - val_recall: 0.6395
Epoch 2/5
43/43 [==============================] - 41s 965ms/step - loss: 0.5499 - accuracy: 0.7422 - recall: 0.7489 - val_loss: 0.5306 - val_accuracy: 0.7723 - val_recall: 0.5816
Epoch 3/5
43/43 [==============================] - 41s 952ms/step - loss: 0.5255 - accuracy: 0.7548 - recall: 0.7370 - val_loss: 0.4717 - val_accuracy: 0.7875 - val_recall: 0.8503
Epoch 4/5
43/43 [==============================] - 41s 942ms/step - loss: 0.5164 - accuracy: 0.7533 - recall: 0.7504 - val_loss: 0.4737 - val_accuracy: 0.7656 - val_recall: 0.8741
Epoch 5/5
43/43 [==============================] - 40s 935ms/step - loss: 0.5104 - accuracy: 0.7533 - recall: 0.7593 - val_loss: 0.4574 - val_accuracy: 0.8027 - val_recall: 0.8639
Total Time Elapsed:  3  minutes  33  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 18s 615ms/step - loss: 0.8800 - accuracy: 0.4921 - recall: 0.3395


## MobileNetV2 -- 15 epochs

In [8]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [9]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [10]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 40s 932ms/step - loss: 0.6082 - accuracy: 0.6917 - recall: 0.7013 - val_loss: 0.5377 - val_accuracy: 0.7218 - val_recall: 0.8299
Epoch 2/15
43/43 [==============================] - 41s 965ms/step - loss: 0.5452 - accuracy: 0.7422 - recall: 0.7385 - val_loss: 0.4932 - val_accuracy: 0.7909 - val_recall: 0.7925
Epoch 3/15
43/43 [==============================] - 41s 946ms/step - loss: 0.5331 - accuracy: 0.7474 - recall: 0.7504 - val_loss: 0.4797 - val_accuracy: 0.7858 - val_recall: 0.7959
Epoch 4/15
43/43 [==============================] - 41s 943ms/step - loss: 0.5201 - accuracy: 0.7600 - recall: 0.7459 - val_loss: 0.5375 - val_accuracy: 0.7167 - val_recall: 0.9082
Epoch 5/15
43/43 [==============================] - 41s 947ms/step - loss: 0.5128 - accuracy: 0.7600 - recall: 0.7474 - val_loss: 0.5184 - val_accuracy: 0.7201 - val_recall: 0.8946
Epoch 6/15
43/43 [==============================] - 40s 941ms/step - loss: 0.5130 - accuracy: 0

In [11]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 19s 617ms/step - loss: 1.2006 - accuracy: 0.4013 - recall: 0.1543


ResNet50 has 25,636,712 trainable parameters, and a top-5 accuracy of 0.921.

## ResNet50 -- 5 epochs

In [4]:
base_model = keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

94773248/94765736 [==============================] - 4s 0us/step


In [5]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [6]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 121s 3s/step - loss: 0.7095 - accuracy: 0.5134 - recall: 0.6196 - val_loss: 0.6939 - val_accuracy: 0.4975 - val_recall: 1.0000
Epoch 2/5
43/43 [==============================] - 121s 3s/step - loss: 0.7012 - accuracy: 0.5052 - recall: 0.5141 - val_loss: 0.6919 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Epoch 3/5
43/43 [==============================] - 121s 3s/step - loss: 0.6989 - accuracy: 0.5156 - recall: 0.5022 - val_loss: 0.6916 - val_accuracy: 0.4958 - val_recall: 1.0000
Epoch 4/5
43/43 [==============================] - 122s 3s/step - loss: 0.6945 - accuracy: 0.5275 - recall: 0.5305 - val_loss: 0.6962 - val_accuracy: 0.4958 - val_recall: 1.0000
Epoch 5/5
43/43 [==============================] - 121s 3s/step - loss: 0.6959 - accuracy: 0.5401 - recall: 0.5156 - val_loss: 0.6856 - val_accuracy: 0.5042 - val_recall: 0.9796
Total Time Elapsed:  10  minutes  19  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 58s 2s/step - loss: 0.6854 - accuracy: 0.6431 - recall: 0.9120


## ResNet50 -- 15 epochs

In [8]:
base_model = keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [9]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [10]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 122s 3s/step - loss: 0.7055 - accuracy: 0.5067 - recall: 0.4368 - val_loss: 0.6933 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Epoch 2/15
43/43 [==============================] - 121s 3s/step - loss: 0.6992 - accuracy: 0.4926 - recall: 0.5736 - val_loss: 0.6928 - val_accuracy: 0.4958 - val_recall: 1.0000
Epoch 3/15
43/43 [==============================] - 121s 3s/step - loss: 0.7001 - accuracy: 0.4978 - recall: 0.4428 - val_loss: 0.6867 - val_accuracy: 0.6796 - val_recall: 0.8946
Epoch 4/15
43/43 [==============================] - 121s 3s/step - loss: 0.6886 - accuracy: 0.5267 - recall: 0.5349 - val_loss: 0.7092 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Epoch 5/15
43/43 [==============================] - 121s 3s/step - loss: 0.6897 - accuracy: 0.5401 - recall: 0.5706 - val_loss: 0.6827 - val_accuracy: 0.6054 - val_recall: 0.2109
Epoch 6/15
43/43 [==============================] - 121s 3s/step - loss: 0.6845 - accuracy: 0.546

In [11]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 57s 2s/step - loss: 0.6835 - accuracy: 0.5502 - recall: 0.6065


While some transfer learning models did have greater evaluation metrics, mainly recall, on average. No models outperformed the custom 2x2C1D with Dropout model from 004_Modeling_Local.ipynb.